This notebook is to load predicted mask for hard-disk and calculate Segmentation evaluation metics.
predicted mask is the output of ScaleNASv2 Test and save predicted mask.ipynb

cp -v /mnt/fast-data/mjc/AutoRECIST/Codes/ScaleNAS/ScaleNASv1/tools/utils_test.py .

In [1]:
import os
site_list = range(1,22)
cache_path = './cache/'
name = 'PDS_AMGEN_20020408_22Cat_test'
cache_filepath = os.path.join(cache_path, name+'_gt_roidb.pkl')


sv_dir = mask_name = 'mask_1988'
sv_path = os.path.join(sv_dir, 'test_val_results')

DL_info = '/mnt/fast-data/mjc/AutoRECIST/Inputs/PDS_AMGEN_20020408_CIA-LAB_Image_And_Contour_2022-01-10.xlsx'
metrics_save_path = 'temp_%s_.csv'%'ScaleNAS_Q5_9Slices_Amgen_Liver'

In [2]:
# coding: utf-8

# # This file is for segmetation metrics evaluation in 3D
# Edited by Jingchen around 06/20/2021
# This file is after ScaleNAS test which save predition into png images.
# This file load png images as predicted contours in 2D
# load cache pkl as gold-standard contours in 2D
# The stack 2D based on dicom-header to get 3D
# Evaluate 3D metics of dice, IoU, over-segmetation, and under-segmetation
# 
%reload_ext autoreload
%autoreload 2


import logging
import numpy as np
import _init_paths

from config import cfg
from config import update_config
from utils_test import *

import cv2
from PIL import Image
import torch.nn.functional as F
from utils.utils import get_confusion_matrix
def convert_name(name):
    new = name.replace('/','_')
    return new

        



In [3]:

print('Loading cached gt_roidb from %s', cache_filepath)
with open(cache_filepath, 'rb') as fp:
    cached_roidb = pickle.load(fp)
    
roidb = cached_roidb

from utils_test import __get_annotation__

all_boxes = [ [ np.zeros((0,5),dtype="float32") for _ in range(len(roidb)) ] for _ in range( cfg.DATASET.NUM_CLASSES) ]
all_segms = [ [ [] for _ in range(len(roidb)) ] for _ in range( cfg.DATASET.NUM_CLASSES) ]

for i in range(len(roidb)):
    if i%500==0:
        print('='*40,i)

    one = roidb[i]
    onename = one['image']
    pred_im = Image.open(os.path.join(sv_path, convert_name(onename) ))
    pred = np.array(pred_im)
    for j in range(cfg.DATASET.NUM_CLASSES):
        mask = np.asarray( pred==j , dtype=np.uint8)
        if np.sum(mask > 0) <= 3 :
            continue
        segmentation, bbox, area = __get_annotation__(mask , xywh = False , bbox_score=True)
        if segmentation and bbox:
            all_segms[j][i] = segmentation
            all_boxes[j][i] = bbox




Loading cached gt_roidb from %s ./cache/PDS_AMGEN_20020408_22Cat_test_gt_roidb.pkl


/mnt/fast-data/mjc/envs/Lesions/lib/python3.6/site-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


======================================== 0
======================================== 500
======================================== 1000
======================================== 1500
======================================== 2000
======================================== 2500
======================================== 3000
======================================== 3500
======================================== 4000
======================================== 4500
======================================== 5000
======================================== 5500
======================================== 6000
======================================== 6500
======================================== 7000
======================================== 7500
======================================== 8000
======================================== 8500
======================================== 9000
======================================== 9500
======================================== 10000
=====================================

In [4]:
D_CT = {}
for i , aroidb in enumerate(roidb):
    dicom_path , png_name = os.path.split(aroidb['image'])
    slice_no , _= os.path.splitext(png_name)
    slice_no = int(slice_no)
    if slice_no != aroidb['slice_no']:
        print(dicom_path,slice_no,aroidb['slice_no'])

    segmentations = {}
    bboxes = {}
    for j in site_list:
        segmentations[j] = all_segms[j][i]
        bboxes[j] = all_boxes[j][i]

    if dicom_path not in D_CT:
        D_CT[dicom_path] = {}
        D_CT[dicom_path][slice_no] = [aroidb , bboxes , segmentations]
    else:
        D_CT[dicom_path][slice_no] = [aroidb , bboxes , segmentations]

In [5]:

import pandas as pd

if DL_info[-4:]=='.csv':
    df = pd.read_csv(DL_info)
else:
    df = pd.read_excel(DL_info)

def pd_str_replace(df , col, ori, new):
    if isinstance(col , str):
        df[col] = df[col].str.replace(ori,new, case = False) 
    elif isinstance(col, list):
        for one in col:
            pd_str_replace(df , one, ori, new)
    else:
        raise('col instence should be str or list')

pd_str_replace(df, ['Image File Path' , 'Contour File Path'], "X:" , "/mnt/W-drive")
pd_str_replace(df, ['Image File Path' , 'Contour File Path'], r"\\" , "/")
pd_str_replace(df, ['Image File Path'], "/mnt/W-drive/ClinicalTrialDone/FNIH_VOLPACK", "/mnt/fast-disk1/mjc/AutoRecist/Inputs")
pd_str_replace(df, ['Image File Path'], "/mnt/W-drive/ClinicalTrials", "/mnt/fast-disk1/mjc/AutoRecist/Inputs")
pd_str_replace(df, ['Contour File Path'], "/mnt/W-drive/ConvWeasisToRaw", "/mnt/fast-disk1/mjc/AutoRecist/Inputs/ConvWeasisToRaw")


sys.path.append('/mnt/fast-data/mjc/DeepLesion/Codes/CollectData/')
from print_weasis_raw_data import WeasisRawDataPrinter
from read_weasis_raw_data import WeasisRawFileReader


import os
import numpy as np
import pydicom
import matplotlib.pylab as plt


D_dir2header_df = {}
def get_dicom_header_df(image_dir , labels = []):
    global D_dir2header_df
    if image_dir in D_dir2header_df:
        return D_dir2header_df[image_dir]

    # image_dir = row['Image File Path']


    labels = ['ImageName','InstanceNumber',
            'BitsAllocated', 'BitsStored', 'Columns', 'HighBit', 
            'ImageOrientationPatient_0', 'ImageOrientationPatient_1', 'ImageOrientationPatient_2',
            'ImageOrientationPatient_3', 'ImageOrientationPatient_4', 'ImageOrientationPatient_5',
            'ImagePositionPatient_0', 'ImagePositionPatient_1', 'ImagePositionPatient_2',
            'Modality', 'PatientID', 'PhotometricInterpretation', 'PixelRepresentation',
            'PixelSpacing_0', 'PixelSpacing_1', 'RescaleIntercept', 'RescaleSlope', 'Rows', 'SOPInstanceUID',
            'SamplesPerPixel', 'SeriesInstanceUID', 'StudyID', 'StudyInstanceUID', 
            'WindowCenter', 'WindowWidth', 
        ] if not labels else labels

    data = {l: [] for l in labels}
    
    ctList = os.listdir(image_dir)
    ctList.sort()

    for image in ctList:
        if '.dcm' not in image:
            continue
        if os.path.getsize(os.path.join(image_dir, image)) < 5*1024:
            print('%s size < 5kb skiped!'%os.path.join(image_dir, image) )
            continue
        data["ImageName"].append(image)

        ds = pydicom.dcmread(os.path.join(image_dir, image))
        for metadata in ds.dir():
            if metadata not in data and metadata not in ['ImageOrientationPatient','ImagePositionPatient','PixelSpacing']:
                continue
            if metadata != "PixelData":
                metadata_values = getattr(ds, metadata)
                if type(metadata_values) == pydicom.multival.MultiValue and metadata not in ["WindowCenter", "WindowWidth"]:
                    for i, v in enumerate(metadata_values):
                        data[f"{metadata}_{i}"].append(v)  
                else:

                    if type(metadata_values) == pydicom.multival.MultiValue and metadata in ["WindowCenter", "WindowWidth"]:
                        data[metadata].append(metadata_values[0])
                    else:
                        
                        if metadata in ['ImageOrientationPatient','ImagePositionPatient','PixelSpacing']:
                            print( 'error of loading key: {}'.format(metadata) )                    
                        else:
                            data[metadata].append(metadata_values)

    df_image = pd.DataFrame(data).set_index("InstanceNumber")
    D_dir2header_df[image_dir] = df_image
    return df_image

In [ ]:
def compare_site(location , site_list):
    for i in site_list:
        if location == ix2labelname(i):
            return True
    return False

In [ ]:
from utils_metrics_3d import *
Metrics_vol = []
Missed_vol = []
previous_CT_ss = ''
for _,row in df.iterrows():
    if not compare_site(row.Location , site_list):
#         only calculate one lesion site
        continue


    df_image = get_dicom_header_df( row['Image File Path'] )
    instanceNumber_list = df_image.index.to_list()
    D_z_index = instanceNumber2Matrix_z_index(instanceNumber_list)

    reader = WeasisRawFileReader()
    weasis_raw_data = reader.read_weasis_raw_file(row['Contour File Path'])

    ss = row['Image File Path']
    ss = ss.replace('/Inputs/' , '/Pngs/')
    if ss not in D_CT:
        print(ss , 'not in D_CT and skip it')
        continue

    mask_vol = initialize_mask_vol(weasis_raw_data , D_z_index)

    slice_list = weasis_raw_data.get_instance_number_array()
    for j, one in enumerate(slice_list):
        file_name = InstanceNumber2file_name(df_image, one)
        file_name = os.path.join( row['Image File Path'] , file_name)   
        mask = weasis_raw_data.get_mask_image_2d(j)
        mask_vol[D_z_index[one]] = mask

    if ss != previous_CT_ss:
        oneCT = D_CT[ss]
        vol_gt , vol_pred = get_gt_and_pred_vols( oneCT, site_list, mask_vol.shape, D_z_index, union_mask=False )
        previous_CT_ss = ss
    hit = vols_seg_results(mask_vol , vol_pred, CTname=row['Contour File Path'], gt_keep_largest=1)
    Metrics_vol.extend(hit)
    
    _n = len(Metrics_vol)
    if _n%100==0 or _n in [1,3,5,10,30,50]:
        print(_, _n)
        df_metrics = pd.DataFrame(Metrics_vol, 
                                  columns = ['file_name','igt','merge','#gt','#pred',
                                             'iou_score', 'dice_score', 'over_seg' , 'under_seg',
                                             'area_gt','area_pred','intersection','union']) 
        df_metrics.to_csv(metrics_save_path)

In [ ]:
df_metrics.to_csv(metrics_save_path)

In [ ]:
print(name)
pd.options.display.float_format = "{:.3f}".format
df_metrics = pd.DataFrame(Metrics_vol, 
                          columns = ['file_name','igt','merge','#gt','#pred',
                                     'iou_score', 'dice_score', 'over_seg' , 'under_seg',
                                     'area_gt','area_pred','intersection','union']) 
df_metrics.describe([.05, .25, .5, .75, .95])

In [ ]:
df_metrics[df_metrics.dice_score>0.25].describe([.5])

In [ ]:
dfinnermerge = pd.merge(df_metrics,df,how='inner',left_on='file_name' , right_on='Contour File Path')

In [ ]:
for col in dfinnermerge.columns.tolist():
    print(col , len(set(dfinnermerge[col].tolist() )) )

In [ ]:
pts = dfinnermerge["Image File Path"].values.tolist()
FPs = []
for onept in list(set(pts)):
    df_onept = dfinnermerge[dfinnermerge["Image File Path"]==onept]
    assert( min(df_onept["#pred"]) == max(df_onept["#pred"]))
    fp = max(df_onept["#pred"])
    FPs.append(fp)

print( len(FPs) , sum(FPs) )

dices = dfinnermerge.dice_score.tolist()

for th in [0, 0.1, 0.2, 0.25, 0.5]:
    TP = [p>th for p in dices ]
    assert( len(TP) == len(dices))
    fprate = ( sum(FPs) - sum(TP) ) / len(FPs)
    print(f"sensitivity is {sum(TP)/len(dices):.3f}({sum(TP)}/{len(dices)}) FP-rate is {fprate:.1f} at threshold {th}")
    

In [ ]:
sum( dfinnermerge['#pred'] )

Cougar
96 pts at baseline timepoints, 209 LN lesions
1489 AI detections
When Dice>0,25 means the lesion is detected
Sensitivity 66%=138/209 
FP rate is 13.2 FPs/CT (too high!)





In [7]:
SAVE_PATH = '/mnt/fast-disk1/mjc/AutoRecist/Outputs/ScaleNAS_Q5_9Slices/'
name = 'Amgen_Allsites'

from utils_metrics_3d import *

SHOW_LABEL = True
SHOW_BOX = False
SHOW_MASK = True
SHOW_UNION_MASK= False
SHOW_MASK_LABEL = True
SHOW_GT_MASK = False

def get_proper_CT_windowing(segmentations):

    if 8 in segmentations and segmentations[8] :
        return -160 , 240
    if 7 in segmentations and segmentations[7] :
        return -160 , 240    
    if 9 in segmentations and segmentations[9] :
        return -1250 , 250
    return None

keys = list(D_CT.keys())

def compute_colors_for_labels(i):
    D = {
        1 : [1, 127, 31] , # abdomen
        2 : [64, 255, 64] , #  abdomen LN
        3 : [255, 64, 255] , # adrenal
        4 : [64, 255, 64] , # axillary LN
        5 : [5, 125, 155] , # bone
        6 : [64, 255, 64] , #  inguinal LN
        7 : [7, 124, 217] , # kidney
        8 : [8, 251, 248] , # liver
        9 : [255, 64, 64] , # lung
        10 : [64, 255, 64] , # mediastinum LN
        11 : [64, 255, 64] , # neck LN
        12 : [12, 249, 117] , # ovary
        13 : [13, 121, 148] , # pancreas
        14 : [64, 255, 64] , # pelvic LN
        15 : [15, 120, 210] , # pelvis
        16 : [16, 247, 241] , # pleural
        17 : [64, 255, 64] , #  retroperitoneal LN
        18 : [18, 246, 48] , # soft tissue
        19 : [19, 118, 79] , # spleen
        20 : [20, 245, 110] , # stomach
        21 : [21, 117, 141] , # thyroid
        }
    return D[i]

for k in keys:
    
    oneCT = remove_single_slice_segms(D_CT[k])
    savepath = os.path.join( SAVE_PATH , 'CTs_%s/%s'%(name,convert_name_compact(k)) )

    print('Image results are saving into {}'.format(savepath))

    if not os.path.exists(savepath):
        os.makedirs(savepath)
    
    for s in oneCT:
        aroidb , bboxes , segmentations = oneCT[s]

        image_path = os.path.join(aroidb['image'])
    
        CT_windowing = get_proper_CT_windowing(segmentations)
        if CT_windowing:
            HU1, HU2 = CT_windowing
        else:
            [HU1, HU2 ] = aroidb['windows']
            

        image = load_image(image_path, HU1, HU2)
        height,width = image.shape
    #     plt.imshow(image)
        image = np.dstack((image,image,image))
        
        if SHOW_MASK:
            for j in site_list:
                if j==0:
                    continue
                contours = segmentations[j]
                colors = compute_colors_for_labels(j)
                label = ix2labelname(j)
                for c in contours:
                    c = np.reshape(c,(-1,2))
                    if c.shape[0]:
                        image = cv2.drawContours(image, [np.int64( c  )], -1, colors, 1)
                        if SHOW_MASK_LABEL:
                            x,y,_,_ =ploy2boxes(c)
                            template = "{}"
                            if len(label)>=9:
                                label=label[:3]+label[-3:]
                
                            s = template.format(label)
                            cv2.putText(
                                image, s, (x, y), cv2.FONT_HERSHEY_SIMPLEX, .5, colors, 1
                            )  
        

        cv2.imwrite( os.path.join(savepath, convert_name_compact( aroidb['image'] ) ) , image )

Image results are saving into /mnt/fast-disk1/mjc/AutoRecist/Outputs/ScaleNAS_Q5_9Slices/CTs_Amgen_Allsites/AMGEN_20020408_BAIJC_D2004_02_27_E20040227_CT_S0002
Image results are saving into /mnt/fast-disk1/mjc/AutoRecist/Outputs/ScaleNAS_Q5_9Slices/CTs_Amgen_Allsites/AMGEN_20020408_BAIJC_D2004_02_27_E20040227_CT_S0003
Image results are saving into /mnt/fast-disk1/mjc/AutoRecist/Outputs/ScaleNAS_Q5_9Slices/CTs_Amgen_Allsites/AMGEN_20020408_BAIJD_D2004_02_07_E20040207_CT_S0002
Image results are saving into /mnt/fast-disk1/mjc/AutoRecist/Outputs/ScaleNAS_Q5_9Slices/CTs_Amgen_Allsites/AMGEN_20020408_BAIJG_D2004_02_02_E20040202_CT_S0005
Image results are saving into /mnt/fast-disk1/mjc/AutoRecist/Outputs/ScaleNAS_Q5_9Slices/CTs_Amgen_Allsites/AMGEN_20020408_BAIJG_D2004_02_02_E20040202_CT_S0013
Image results are saving into /mnt/fast-disk1/mjc/AutoRecist/Outputs/ScaleNAS_Q5_9Slices/CTs_Amgen_Allsites/AMGEN_20020408_BAIJH_D2004_02_02_E20040202_CT_S0003
Image results are saving into /mnt/fast-